In [1]:
import pandas as pd
import numpy as np
import re
import sys
import time


start = time.time()
new_cdr_data = pd.ExcelFile('dboutput100k.xlsx')
cdr_new = new_cdr_data.parse('dboutput')
cdr_for_analysis = cdr_new.head(100)

In [2]:
df = cdr_for_analysis[['xbin', 'ybin']]
measurmeents = cdr_for_analysis['measurements']

fd = cdr_for_analysis[['xbin','ybin','measurements']]
fd_meaurements = fd['measurements']

In [3]:
def tidy_split(df, column, sep='|', keep=False):
    """
    Split the values of a column and expand so the new DataFrame has one split
    value per row. Filters rows where the column is missing.

    Params
    ------
    df : pandas.DataFrame
        dataframe with the column to split and expand
    column : str
        the column to split and expand
    sep : str
        the string used to split the column's values
    keep : bool
        whether to retain the presplit value as it's own row

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns as `df`.
    """
    indexes = list()
    new_values = list()
    df = df.dropna(subset=[column])
    for i, presplit in enumerate(df[column].astype(str)):
        values = presplit.split(sep)
        if keep and len(values) > 1:
            indexes.append(i)
            new_values.append(presplit)
        for value in values:
            indexes.append(i)
            new_values.append(value)
    new_df = df.iloc[indexes, :].copy()
    new_df[column] = new_values
    new_df 
    return new_df

In [4]:
final = tidy_split(fd, 'measurements', sep=('},{'))

In [5]:
final

,xbin,ybin,measurements
0,703000.0,3661850.0,"[{ltecelloid:10425,eventtype:1,rsrp:-93.5,phys..."
0,703000.0,3661850.0,"ltecelloid:10425,eventtype:5,rsrp:-94.5,physic..."
0,703000.0,3661850.0,"rsrp:-93.5,rsrq:-17.75,eventtype:5,physicalcel..."
0,703000.0,3661850.0,"rsrp:-92.5,rsrq:-16.25,eventtype:5,physicalcel..."
1,736600.0,3636800.0,"[{ltecelloid:8997,eventtype:9,rsrp:-107.5,phys..."
...,...,...,...
97,703850.0,3639100.0,"[{ltecelloid:8635,eventtype:9,rsrp:-96.5,physi..."
97,703850.0,3639100.0,"ltecelloid:8629,eventtype:9,rsrp:-108.5,physic..."
98,702800.0,3662800.0,"[{ltecelloid:13975,eventtype:9,rsrp:-103.5,phy..."
98,702800.0,3662800.0,"rsrp:-113.5,rsrq:-19.5,eventtype:9,physicalcel..."


In [6]:
final['measurements'] = final['measurements'].map(lambda x: x.lstrip('+{[:').rstrip('rsrp'))

In [7]:
che=final.reset_index()

In [8]:
che


,index,xbin,ybin,measurements
0,0,703000.0,3661850.0,"ltecelloid:10425,eventtype:1,rsrp:-93.5,physic..."
1,0,703000.0,3661850.0,"ltecelloid:10425,eventtype:5,rsrp:-94.5,physic..."
2,0,703000.0,3661850.0,"rsrp:-93.5,rsrq:-17.75,eventtype:5,physicalcel..."
3,0,703000.0,3661850.0,"rsrp:-92.5,rsrq:-16.25,eventtype:5,physicalcel..."
4,1,736600.0,3636800.0,"ltecelloid:8997,eventtype:9,rsrp:-107.5,physic..."
...,...,...,...,...
322,97,703850.0,3639100.0,"ltecelloid:8635,eventtype:9,rsrp:-96.5,physica..."
323,97,703850.0,3639100.0,"ltecelloid:8629,eventtype:9,rsrp:-108.5,physic..."
324,98,702800.0,3662800.0,"ltecelloid:13975,eventtype:9,rsrp:-103.5,physi..."
325,98,702800.0,3662800.0,"rsrp:-113.5,rsrq:-19.5,eventtype:9,physicalcel..."


In [9]:
foo = lambda x: pd.Series([i for i in reversed(x.split(','))])
rev = che['measurements'].apply(foo)
print (rev)

                           0            1                          2   \
0      trackingareacode:27200  rsrq:-12.25  uplinkchannelnumber:20450   
1      trackingareacode:27200  rsrq:-13.25  uplinkchannelnumber:20450   
2          physicalcellid:407  eventtype:5                rsrq:-17.75   
3        physicalcellid:406}]  eventtype:5                rsrq:-16.25   
4      trackingareacode:27147  rsrq:-13.25  uplinkchannelnumber:20549   
..                        ...          ...                        ...   
322    trackingareacode:27186  rsrq:-11.75  uplinkchannelnumber:20549   
323  trackingareacode:27186}]   rsrq:-9.75  uplinkchannelnumber:19000   
324    trackingareacode:27200  rsrq:-12.75  uplinkchannelnumber:23780   
325      physicalcellid:371}]  eventtype:9                 rsrq:-19.5   
326  trackingareacode:27148}]   rsrq:-6.75  uplinkchannelnumber:20450   

                  3                           4                    5   \
0    enodeb:DXL03207  downlinkchannelnumber:2450  

In [10]:
rev.rename(columns={0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',8:'rsrp',9:'j',10:'Itecelloid'},inplace=True)
rev = rev[['a','b','c','d','e','f','g','h','rsrp','j','Itecelloid']]
#print (rev)

rev

,a,b,c,d,e,f,g,h,rsrp,j,Itecelloid
0,trackingareacode:27200,rsrq:-12.25,uplinkchannelnumber:20450,enodeb:DXL03207,downlinkchannelnumber:2450,label:DXL03207_8A_1,frequencyband:850,physicalcellid:87,rsrp:-93.5,eventtype:1,ltecelloid:10425
1,trackingareacode:27200,rsrq:-13.25,uplinkchannelnumber:20450,enodeb:DXL03207,downlinkchannelnumber:2450,label:DXL03207_8A_1,frequencyband:850,physicalcellid:87,rsrp:-94.5,eventtype:5,ltecelloid:10425
2,physicalcellid:407,eventtype:5,rsrq:-17.75,rsrp:-93.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,physicalcellid:406}],eventtype:5,rsrq:-16.25,rsrp:-92.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,trackingareacode:27147,rsrq:-13.25,uplinkchannelnumber:20549,enodeb:DXL02978,downlinkchannelnumber:2549,label:DXL02978_8B_2,frequencyband:850,physicalcellid:128,rsrp:-107.5,eventtype:9,ltecelloid:8997
...,...,...,...,...,...,...,...,...,...,...,...
322,trackingareacode:27186,rsrq:-11.75,uplinkchannelnumber:20549,enodeb:DXL03017,downlinkchannelnumber:2549,label:DXL03017_8A_2,frequencyband:850,physicalcellid:165,rsrp:-96.5,eventtype:9,ltecelloid:8635
323,trackingareacode:27186}],rsrq:-9.75,uplinkchannelnumber:19000,enodeb:DXL03017,downlinkchannelnumber:1000,label:DXL03017_9A_1,frequencyband:1900,physicalcellid:297,rsrp:-108.5,eventtype:9,ltecelloid:8629
324,trackingareacode:27200,rsrq:-12.75,uplinkchannelnumber:23780,enodeb:DXL00278,downlinkchannelnumber:5780,label:DXL00278_7C_1,frequencyband:700,physicalcellid:497,rsrp:-103.5,eventtype:9,ltecelloid:13975
325,physicalcellid:371}],eventtype:9,rsrq:-19.5,rsrp:-113.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
che['itecelloid'] = rev['Itecelloid'].values 
che['rsrp'] = rev['rsrp'].values

In [12]:
che['itecelloid'] = che['itecelloid'].astype(str).str.replace('\D+', '')
che['rsrp'] = che['rsrp'].astype(str).str.replace('rsrp:','')

In [13]:
che.dtypes

index             int64
xbin            float64
ybin            float64
measurements     object
itecelloid       object
rsrp             object
dtype: object

In [14]:
che= che.dropna(subset=['rsrp'])

In [15]:
final_che= che[["xbin","ybin","itecelloid","rsrp"]]
final_che


,xbin,ybin,itecelloid,rsrp
0,703000.0,3661850.0,10425,-93.5
1,703000.0,3661850.0,10425,-94.5
2,703000.0,3661850.0,,nan
3,703000.0,3661850.0,,nan
4,736600.0,3636800.0,8997,-107.5
...,...,...,...,...
322,703850.0,3639100.0,8635,-96.5
323,703850.0,3639100.0,8629,-108.5
324,702800.0,3662800.0,13975,-103.5
325,702800.0,3662800.0,,nan


In [16]:
A = final_che[final_che['itecelloid'].astype(bool)]


In [17]:
A['rsrp'].mean()

TypeError: Could not convert -93.5-94.5-107.5-115.5-113.5-110.5-114.5-116.5-106.5-110.5-103.5-124.5-87.5-97.5-99.5-104.5-104.5-101.5-98.5-82.5-108.5-101.5-81.5-107.5-105.5-112.5-100.5-87.5-104.5-85.5-110.5-104.5-107.5-109.5-97.5-81.5-101.5-84.5-109.5-102.5-113.5-113.5-113.5-101.5-88.5-98.5-85.5-102.5-112.5-115.5-113.5-102.5-102.5-85.5-103.5-111.5-111.5-114.5-101.5-103.5-103.5-110.5-112.5-115.5-103.5-103.5-102.5-111.5-112.5-115.5-102.5-102.5-102.5-113.5-111.5-112.5-102.5-90.5-102.5-112.5-113.5-116.5-102.5-102.5-84.5-83.5-104.5-83.5-86.5-102.5-102.5-113.5-112.5-112.5-117.5-102.5-89.5-94.5-92.5-102.5-112.5-115.5-111.5-105.5-115.5-105.5-116.5-84.5-91.5-112.5-117.5-112.5-118.5-112.5-117.5-112.5-118.5-111.5-114.5-116.5-113.5-117.5-115.5-120.5-122.5-111.5-116.5-112.5-123.5-114.5-125.5-104.5-105.5-107.5-104.5-96.5-85.5-90.5-89.5-98.5-97.5-101.5-103.5-97.5-86.5-83.5-105.5-109.5-109.5-110.5-115.5-112.5-110.5-98.5-114.5-106.5-93.5-79.5-86.5-79.5-91.5-85.5-94.5-76.5-85.5-78.5-86.5-107.5-106.5-102.

In [18]:
grouped = A.groupby(['xbin','ybin'])

In [19]:
grouped = A.groupby(['xbin','ybin'])

for name,group in grouped:
    print(name)
    #print(group)
    group['rsrp'] = pd.to_numeric(group['rsrp'])
    g = group.groupby(['itecelloid'],as_index=False).agg({'rsrp': 'mean'})
    print (g)
    if not g.empty:
       print ("Data available")
       maxi = g.loc[g['rsrp'].idxmax()]
       print(maxi)
   #if not g.

(701650.0, 3650850.0)
  itecelloid   rsrp
0      14779 -113.5
Data available
itecelloid    14779
rsrp         -113.5
Name: 0, dtype: object
(702700.0, 3630100.0)
  itecelloid        rsrp
0      24917 -110.166667
1      24918 -110.833333
2      38208 -110.500000
Data available
itecelloid      24917
rsrp         -110.167
Name: 0, dtype: object
(702800.0, 3662800.0)
  itecelloid        rsrp
0      13975 -103.500000
1      13976  -87.071429
2      37852 -104.500000
3      47572  -90.500000
Data available
itecelloid      13976
rsrp         -87.0714
Name: 1, dtype: object
(703000.0, 3661850.0)
  itecelloid  rsrp
0      10425 -94.0
Data available
itecelloid    10425
rsrp            -94
Name: 0, dtype: object
(703300.0, 3648450.0)
  itecelloid   rsrp
0      27618 -103.5
1      45025 -101.5
Data available
itecelloid    45025
rsrp         -101.5
Name: 1, dtype: object
(703550.0, 3646850.0)
  itecelloid   rsrp
0      28522 -111.0
1      28523 -108.0
Data available
itecelloid    28523
rsrp        

In [2]:
print (c)

5
